# Capstone Project - Moving to New York City

Let's load all necessary libraries:

In [54]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np # library to handle data in a vectorized manner
#import random # library for random number generation

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# importing libraries for hierarchical clustering
import pylab
import scipy.cluster.hierarchy

print('All done!')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

All done!


## 1. Introduction/Business Problem

Let us assume I live in Zurich at Helvetiaplatz, a multicultural area of Zurich, and found a new job in Manhattan, New York. However, I really like the spirit of the place I am currently living and therefore I am looking for a similar area in terms of culture, food, etc. to live in Manhattan. Which Neighborhoods in Manhattan should I consider to live in?

This report uses a k-means approach to find similar neighborhoods based on the 10 most common venues in each neighborhood. It can be used to find neighborhoods in a foreign city which are similar to a given area.

## 2. Data

For Manhattan I use the data set from https://geo.nyu.edu/catalog/nyu_2451_34572 which contains all boroughs and neighborhoods of NYC as well as latitude and longitute of each neighborhood. As in the Lab, I use the data set provided on https://cocl.us/new_york_dataset.

I use the geolocator to find the coordinates of Zurich Helvetiaplatz.

To get data about the venues around Helvetiaplatz in Zurich as well as around all neighbourhoods in Manhattan, I use the Foursquare location data. Foursquare location data contains venues and their categories as well as coordinates.




### Download and preparation of the data about Manhattan's neighborhoods

In [55]:
# Load the data
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

# Load the data
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
# Extract the list of neighborhoods out of the features key
neighborhoods_data = newyork_data['features']

# Define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# Instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

# Let's only extract neighborhoods from Manhattan
neighborhoods = neighborhoods[neighborhoods.Borough == 'Manhattan']

# Let's have a look at the data
neighborhoods.head()

Data downloaded!


,Borough,Neighborhood,Latitude,Longitude
6,Manhattan,Marble Hill,40.876551,-73.910660
100,Manhattan,Chinatown,40.715618,-73.994279
101,Manhattan,Washington Heights,40.851903,-73.936900
102,Manhattan,Inwood,40.867684,-73.921210
103,Manhattan,Hamilton Heights,40.823604,-73.949688


### Preparation of the data about Zurich Helvetiaplatz

Next, I calculate the coordinates of Helvetiaplatz, then I define borough as 'Kreis 4' and neighborhood as 'Langstrasse'.

In [56]:
address = 'Helvetiaplatz, Zürich'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
Helvetiapaltz_latitude = location.latitude
Helvetiapaltz_longitude = location.longitude
#print(latitude, longitude)

Helvetiapaltz_borough = 'Kreis 4'
Helvetiapaltz_neighborhood = 'Langstrasse'

### Concatenate the two data sets

The next step is to add the data from Zurich Helvetiaplatz to the neighborhood data of Manhattan:

In [57]:
neighborhoods = neighborhoods.append({'Borough': Helvetiapaltz_borough,
                                          'Neighborhood': Helvetiapaltz_neighborhood,
                                          'Latitude': Helvetiapaltz_latitude,
                                          'Longitude': Helvetiapaltz_longitude}, ignore_index=True)

# Let's have a look at the data
neighborhoods.tail()

,Borough,Neighborhood,Latitude,Longitude
36,Manhattan,Tudor City,40.746917,-73.971219
37,Manhattan,Stuyvesant Town,40.731000,-73.974052
38,Manhattan,Flatiron,40.739673,-73.990947
39,Manhattan,Hudson Yards,40.756658,-74.000111
40,Kreis 4,Langstrasse,47.376083,8.526900


## 3. Methodology

### Idea

To check similarities between objects, clustering is a common procedure. The idea is to cluster all neighborhoods from Manhattan together with Langstrasse. Those neighborhoods which are in the same cluster as Langstrasse are considered similar to Langstrasse and are therefore eligible for living. I choose the most popular clustering algorithm, k-means. In case results would not be satisfactory, hierarchical clustering could be chosen to find out more about similarity among neighborhoods.

The features used to cluster neighborhoods are given by venue categories from Foursquare: For each venue category within a neighborhood, the portion of venues is calculated.

### Explore Neighborhoods in Manhattan together with Langstrasse

In this section, we are using the Foursquare API to explore the neighborhoods and segment them.

Let's define Foursquare Credentials and Version

In [58]:
CLIENT_ID = '********************************************' # your Foursquare ID
CLIENT_SECRET = '********************************************' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

Let's create a function to extract venues (including venue category) from each neighborhood in the data set

In [59]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create a new dataframe called *neighborhood_venues* which contains a maximum of 100 venues for each neighborhood:

In [60]:
LIMIT = 100

neighborhood_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards
Langstrasse


In [61]:
# Let's check the size of the data set and have a look at it
print(neighborhood_venues.shape)
neighborhood_venues.tail()

(3408, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3403,Langstrasse,47.376083,8.5269,Metropol,47.373692,8.530432,Movie Theater
3404,Langstrasse,47.376083,8.5269,Neela,47.372413,8.524280,Sri Lankan Restaurant
3405,Langstrasse,47.376083,8.5269,Activ Fitness,47.373599,8.530615,Gym
3406,Langstrasse,47.376083,8.5269,Gamper Restaurant,47.379262,8.525264,Restaurant
3407,Langstrasse,47.376083,8.5269,Restaurant Pflug,47.378946,8.524999,Restaurant


Let's check how many venues were returned for each neighborhood

In [62]:
neighborhood_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,45,45,45,45,45,45
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,45,45,45,45,45,45
East Village,100,100,100,100,100,100


Let's find out how many unique categories can be curated from all the returned venues

In [63]:
print('There are {} uniques categories.'.format(len(neighborhood_venues['Venue Category'].unique())))

There are 334 uniques categories.


### Analyze Each Neighborhood

In [64]:
# one hot encoding
neighborhood_onehot = pd.get_dummies(neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhood_onehot['Neighborhood'] = neighborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhood_onehot.columns[-1]] + list(neighborhood_onehot.columns[:-1])
neighborhood_onehot = neighborhood_onehot[fixed_columns]

# Let's print the shape
print('Shape: ',neighborhood_onehot.shape)

# Let's have a look at the data set
neighborhood_onehot.tail()

Shape:  (3408, 335)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Ski Sh

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [65]:
neighborhood_grouped = neighborhood_onehot.groupby('Neighborhood').mean().reset_index()
neighborhood_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Ski Sh

### Cluster Neighborhoods

I choose an initial k equal to 5 and check in which cluster Langstrasse (or Helvetiaplatz) belongs. It actually turns out, that chosing a larger k results in a situation where Langstrasse is it's own cluster, which is not in our favor. Hence, k = 5 is the best choice we can make using k-means.

In [66]:
# set number of clusters
kclusters = 5

neighborhood_grouped_clustering = neighborhood_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhood_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 1, 2, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well each neighborhood:

In [67]:
# add clustering labels to Neighborhoods
neighborhoods_venues_sorted = neighborhood_grouped[['Neighborhood']]
neighborhood_clusters = neighborhoods_venues_sorted.insert(1, 'Cluster Labels', kmeans.labels_)

# Merge neighborhoods and clusters to latitude/longitude for each neighborhood
neighborhoods_merged = neighborhoods
neighborhoods_merged = neighborhoods_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

neighborhoods_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,Manhattan,Marble Hill,40.876551,-73.910660,0
1,Manhattan,Chinatown,40.715618,-73.994279,1
2,Manhattan,Washington Heights,40.851903,-73.936900,2
3,Manhattan,Inwood,40.867684,-73.921210,2
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2
5,Manhattan,Manhattanville,40.816934,-73.957385,0
6,Manhattan,Central Harlem,40.815976,-73.943211,1
7,Manhattan,East Harlem,40.792249,-73.944182,2
8,Manhattan,Upper East Side,40.775639,-73.960508,1
9,Manhattan,Yorkville,40.775930,-73.947118,0


As can be seen above, Langstrasse is in Cluster Label 0, together with 10 neighborhoods of Manhatten. So it shouldn't be a problem to find a place to live similar to the current home.

Let's plot the neighborhoods colored according to their Cluster Labels.

In [68]:
# get coordinates of Manhattan to be able to center the map correctly
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_merged['Latitude'], neighborhoods_merged['Longitude'], neighborhoods_merged['Neighborhood'], neighborhoods_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


## 4. Results

Let's display the ten neighborhoods which are most similar to Langstrasse in Zürich and show them on a map:

In [69]:
most_similar_neighborhoods = neighborhoods_merged[(neighborhoods_merged['Cluster Labels']==0) & 
                                                  (neighborhoods_merged.Borough == 'Manhattan')].reset_index(drop=True).drop('Cluster Labels',1)
most_similar_neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Manhattanville,40.816934,-73.957385
2,Manhattan,Yorkville,40.775930,-73.947118
3,Manhattan,Lenox Hill,40.768113,-73.958860
4,Manhattan,Roosevelt Island,40.762160,-73.949168
5,Manhattan,Upper West Side,40.787658,-73.977059
6,Manhattan,East Village,40.727847,-73.982226
7,Manhattan,Gramercy,40.737210,-73.981376
8,Manhattan,Financial District,40.707107,-74.010665
9,Manhattan,Carnegie Hill,40.782683,-73.953256


In [70]:
# get coordinates of Manhattan to be able to center the map correctly
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)



# add markers to the map
markers_colors = []
for lat, lon, poi in zip(most_similar_neighborhoods['Latitude'], most_similar_neighborhoods['Longitude'],most_similar_neighborhoods['Neighborhood']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7
    ).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


## 5. Discussion

Out of the 40 neighborhoods of Manhattan, we found 10 neighborhoods similar to Langstrasse in Zurich. This is a good base to start searching for a flat based on the exact place of work as well as current availability and prices of flats.

I think this is a good result as it leaves enough places to look for a flat and can now be narrowed down by other criteria like time to commute, prices and availability of flats, noise, pollution, crime rate, etc.

Hierarchical clustering could have been used alternatively, especially if the amount of similar neighborhoods would have been too large.

## 6. Conclusion

Marble Hill, Manhattanville, Yorkville, Lenox Hill, Roosevelt Island, Upper West Side, East Village, Gramercy, Financial District and Carnegie Hill are suggested as neighborhoods to live in Manhattan which are most similar to Langstrasse (Helvetiaplatz) in Zurich in terms of nearby venues.